In [1]:
!pip install llama-index llama-index-core llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes wikipedia

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
 

In [1]:
import torch
import wikipedia
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 1. Configuración del Modelo Local (Hugging Face)
# Usamos Qwen 1.5B porque es rápido, entiende español y no crashea la RAM de Colab.
print("⚙️ Cargando modelo local (esto toma unos segundos)...")

Settings.llm = HuggingFaceLLM(
    model_name="Qwen/Qwen2.5-1.5B-Instruct",
    tokenizer_name="Qwen/Qwen2.5-1.5B-Instruct",
    context_window=4096,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.1, "do_sample": False},
    device_map="auto", # Usa tu GPU automáticamente
)

# 2. Configuración de Embeddings Local
# BAAI/bge-m3 es excelente para español e inglés.
print("⚙️ Cargando embeddings locales...")
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

# 3. Descarga de Datos (Wikipedia)
wikipedia.set_lang("es")
temas = [
    "Inteligencia artificial",         # Doc 1
    "Red neuronal artificial",         # Doc 2
    "Aprendizaje profundo",            # Doc 3
    "Procesamiento de lenguaje natural", # Doc 4
    "Visión artificial"                # Doc 5
]
docs = []

print("\n📚 Descargando documentos...")
for tema in temas:
    try:
        content = wikipedia.page(tema).content
        docs.append(Document(text=content, metadata={"title": tema}))
        print(f"   -> Descargado: {tema}")
    except:
        pass

# 4. Pipeline RAG
if docs:
    print("\n🧠 Indexando en memoria (Vector Store)...")
    index = VectorStoreIndex.from_documents(docs)
    query_engine = index.as_query_engine()

    # 5. Prueba
    pregunta = "¿Qué es una red neuronal y para qué sirve?"
    print(f"\n🗣️ Pregunta: {pregunta}")
    print("-" * 40)

    response = query_engine.query(pregunta)
    print(f"🤖 Respuesta (Local):\n{response}")

    print("\n🔍 Evidencia:")
    for node in response.source_nodes:
        print(f"   📄 Fuente: {node.metadata['title']}")
else:
    print("Error descargando documentos.")

⚙️ Cargando modelo local (esto toma unos segundos)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]


📚 Descargando documentos...
   -> Descargado: Inteligencia artificial
   -> Descargado: Red neuronal artificial
   -> Descargado: Aprendizaje profundo

🧠 Indexando en memoria (Vector Store)...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🗣️ Pregunta: ¿Qué es una red neuronal y para qué sirve?
----------------------------------------
🤖 Respuesta (Local):
 Una red neuronal artificial (NN) es un modelo dentro de los llamados sistemas conexionistas, inspirado en la estructura y función de las redes neuronales biológicas en los cerebros animales. Una ANN consta de unidades o nodos conectados llamados neuronas artificiales, que modelan vagamente las neuronas del cerebro. Estas están conectadas por bordes, que modelan las sinapsis del cerebro. Cada neurona artificial recibe "señales" de las neuronas conectadas, luego las procesa y envía una señal a otras neuronas conectadas. La "señal" es un número real, y la salida de cada neurona se calcula mediante una función no lineal de la suma de sus entradas, llamada función de activación. La fuerza de la señal en cada conexión está determinada por un peso, que se ajusta durante el proceso de aprendizaje. Las redes neuronales se entrenan típicamente a través de la minimización de rie

In [4]:
import os
import wikipedia
from llama_index.core import Document

# 1. Configuración de temas (Tus 5 documentos obligatorios)
wikipedia.set_lang("es")
temas = [
    "Inteligencia artificial",         # Doc 1
    "Red neuronal artificial",         # Doc 2
    "Aprendizaje profundo",            # Doc 3
    "Procesamiento de lenguaje natural", # Doc 4
    "Visión artificial"                # Doc 5
]

docs = []

# 2. Crear una carpeta física para guardar los archivos
carpeta_evidencia = "mis_documentos_rag"
if not os.path.exists(carpeta_evidencia):
    os.makedirs(carpeta_evidencia)
    print(f"📂 Carpeta creada: {carpeta_evidencia}")

print("\n⬇️ Descargando y guardando archivos...")

for tema in temas:
    try:
        # Obtener contenido de Wikipedia
        pagina = wikipedia.page(tema)
        contenido = pagina.content

        # Crear nombre de archivo seguro (sin espacios)
        nombre_archivo = f"{tema.replace(' ', '_')}.txt"
        ruta_archivo = os.path.join(carpeta_evidencia, nombre_archivo)

        # 3. GUARDAR EL ARCHIVO EN EL DISCO (.txt)
        with open(ruta_archivo, "w", encoding="utf-8") as f:
            f.write(f"Fuente: {pagina.url}\n\n") # Agregamos la URL al inicio
            f.write(contenido)

        # Agregamos a la lista de LlamaIndex para el RAG
        docs.append(Document(
            text=contenido,
            metadata={"title": tema, "filename": nombre_archivo}
        ))

        print(f"   ✅ Guardado: {ruta_archivo}")

    except Exception as e:
        print(f"   ❌ Error con {tema}: {e}")

print(f"\n📚 Total documentos listos: {len(docs)}")
print("👉 Revisa la carpeta en el menú de la izquierda para ver tus .txt")

📂 Carpeta creada: mis_documentos_rag

⬇️ Descargando y guardando archivos...
   ✅ Guardado: mis_documentos_rag/Inteligencia_artificial.txt
   ✅ Guardado: mis_documentos_rag/Red_neuronal_artificial.txt
   ✅ Guardado: mis_documentos_rag/Aprendizaje_profundo.txt
   ✅ Guardado: mis_documentos_rag/Procesamiento_de_lenguaje_natural.txt
   ✅ Guardado: mis_documentos_rag/Visión_artificial.txt

📚 Total documentos listos: 5
👉 Revisa la carpeta en el menú de la izquierda para ver tus .txt


In [3]:
# --- CONFIGURACIÓN DE PREGUNTAS SENCILLAS ---
nuevas_preguntas = [
    "¿Qué es la inteligencia artificial en pocas palabras?",
    "¿En qué se inspiran las redes neuronales artificiales?",
    "Menciona un ejemplo práctico de uso del aprendizaje profundo.",
    "¿Cuál es la función básica de una neurona artificial?",
    "¿Qué diferencia hay entre una red neuronal y el cerebro humano?"
]

# Nombre del archivo donde guardaremos todo
nombre_archivo = "evidencia_respuestas.txt"

print(f"💾 Comenzando a responder y guardar en '{nombre_archivo}'...\n")

# Abrimos el archivo en modo escritura ("w")
with open(nombre_archivo, "w", encoding="utf-8") as archivo:
    # Escribimos un encabezado bonito
    archivo.write("=== REPORTE DE EVIDENCIA RAG (Hugging Face Local) ===\n")
    archivo.write("Modelo: Qwen/Qwen2.5-1.5B-Instruct\n")
    archivo.write("Fuentes: Wikipedia (IA, Redes Neuronales, Deep Learning)\n")
    archivo.write("=" * 60 + "\n\n")

    for i, pregunta in enumerate(nuevas_preguntas, 1):
        print(f"procesando pregunta {i}/{len(nuevas_preguntas)}: {pregunta}...")

        # 1. Consultar al modelo
        respuesta = query_engine.query(pregunta)
        texto_respuesta = str(respuesta).strip()

        # 2. Obtener los nombres de los documentos usados (fuentes)
        fuentes = []
        for node in respuesta.source_nodes:
            titulo = node.metadata.get('title', 'Desconocido')
            if titulo not in fuentes: # Evitar repetidos
                fuentes.append(titulo)
        fuentes_str = ", ".join(fuentes)

        # 3. Guardar en el archivo TXT
        archivo.write(f"PREGUNTA {i}: {pregunta}\n")
        archivo.write(f"RESPUESTA: {texto_respuesta}\n")
        archivo.write(f"FUENTES: {fuentes_str}\n")
        archivo.write("-" * 50 + "\n\n")

print(f"\n✅ ¡Listo! Todas las respuestas se guardaron en {nombre_archivo}")
print("📂 Busca el archivo en la carpeta de archivos de Colab (icono de carpeta a la izquierda).")

💾 Comenzando a responder y guardar en 'evidencia_respuestas.txt'...

procesando pregunta 1/5: ¿Qué es la inteligencia artificial en pocas palabras?...
procesando pregunta 2/5: ¿En qué se inspiran las redes neuronales artificiales?...
procesando pregunta 3/5: Menciona un ejemplo práctico de uso del aprendizaje profundo....
procesando pregunta 4/5: ¿Cuál es la función básica de una neurona artificial?...
procesando pregunta 5/5: ¿Qué diferencia hay entre una red neuronal y el cerebro humano?...

✅ ¡Listo! Todas las respuestas se guardaron en evidencia_respuestas.txt
📂 Busca el archivo en la carpeta de archivos de Colab (icono de carpeta a la izquierda).


In [2]:
import torch
print(f"¿Hay GPU disponible?: {torch.cuda.is_available()}")
# Debe decir: True

¿Hay GPU disponible?: False
